In [1]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from dotenv import load_dotenv
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, ListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts.few_shot import FewShotPromptTemplate
import re
import os

In [2]:
from vector_database_kh import create_milvus_db


c:\GenAIProject\GenAIGroupProject\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
from pymilvus import (
    connections,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility,
)

In [10]:
from sentence_transformers import SentenceTransformer

In [3]:
vector_store = create_milvus_db()

c:\GenAIProject\GenAIGroupProject\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Index and embeddings have been successfully loaded from data/annual_filings_model_state.pkl
Index and embeddings have been successfully inserted into your Milvus Collection


In [11]:
vector_store.load()

In [9]:
connections.connect(host="localhost", port="19530")

In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\GenAIProject\GenAIGroupProject\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
query_qn = "How many iphones were sold in 2021"
embedded_query_qn = model.encode(query_qn)

In [17]:
search_params = {
    "metric_type": "L2"
}

result = vector_store.search(
    [embedded_query_qn.tolist()], # a single query must still be enclosed in a list
    "embedding",
    search_params,
    limit=5
)

for hit in result:
    print(hit)
    print(hit[0])
    print(hit[0].id)

['id: 1935, distance: 0.6662631034851074, entity: {}', 'id: 2145, distance: 0.7552067041397095, entity: {}', 'id: 1700, distance: 0.7903969883918762, entity: {}', 'id: 397, distance: 0.7917543649673462, entity: {}', 'id: 1978, distance: 0.7963277101516724, entity: {}']
id: 1935, distance: 0.6662631034851074, entity: {}
1935


In [21]:
search_result = vector_store.query(expr=f"pk == {1935}", output_fields=["*"])

# Print the search result
print(search_result)

[{'pk': 1935, 'embedding': [-0.06728231, -0.09216395, 0.04787752, -0.03930773, 0.019780798, 0.06657909, 0.012856519, 0.020835219, 0.017616091, 0.018569017, 0.075725965, 0.033424772, 0.025746675, -0.017526561, 0.07075583, -0.04813127, 0.052598156, -0.070672676, -0.07139801, -0.009615908, 0.050134078, 0.06436616, 0.008036388, 0.04388648, 0.05663751, 0.032957144, -0.10109895, -0.061987128, -0.070719786, -0.008989388, -0.0072938926, 0.053331256, 0.035424292, 0.08004637, -0.0049086073, -0.15163846, -0.029077401, 0.010319345, 0.0041560233, -0.020652303, -0.029792583, -0.08916836, -0.049034085, 0.104900874, 0.053027555, 0.0041458793, -0.07386805, 0.022767719, -0.022616688, 0.092046306, -0.002511514, 0.046232514, -0.009588766, 0.07673581, -0.0008623279, 0.009017158, -0.06038827, 0.019698802, 0.064244375, 0.0404921, 0.08551747, -0.066516764, 0.0008964158, 0.00067777035, 0.024973558, -0.02927507, -0.052622207, -0.041493125, -0.01517896, 0.005326369, -0.01629912, -0.018979156, 0.05208672, 0.03905

In [22]:
decoded_result = model.decode(search_result)

AttributeError: 'SentenceTransformer' object has no attribute 'decode'

In [18]:
print(f"Query: {query_qn}")
print(f"Response: {id}")

Query: How many iphones were sold in 2021


NameError: name 'df' is not defined

In [4]:
load_dotenv()

def get_numbers(query: str):
    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)
    template = """
    You are smart agent that can idenify all the client ids to solve the query. 
    If no start point for a client is mentioned, assume client 1 is the first client.
    id. From the query, return all the ids that the query refers to as a list of integers.
    {format_instructions}
    {query}
    """

    class Numbers(BaseModel):
        numbers: str = Field(description="should say 'the numbers extracted from the query'")
        relevant_numbers: list = Field(description="should be a list of all the relevant numbers the llm has picked up")

    parser = JsonOutputParser(pydantic_object=Numbers)

    prompt = PromptTemplate(
        input_variables = ['query'],
        template=template,
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser
    ans = chain.invoke(query)
    return ans['relevant_numbers']
        

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to that query.
    """
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(query : str):
    """
    Uses the content from the file path to create embeddings on the inputted data. Using the query
    provided, perform a similarity search on the query and return the output. If you can not provide a response,
    say I don't know rather than providing an incorrect response.
    """

    #file_path = r'C:\GenAI\Final Project\GenAIGroupProject\apple.txt'
    file_path = r'./apple.txt'
    file_path = file_path.strip()
    loader = TextLoader(file_path, encoding= 'utf-8')
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    docs = vectorstore.similarity_search(query)
    return docs[0].page_content


@tool
def query_database(query : str):
    """
    This function queries the SQL database based on the query that the user inputs.
    This will provide insight into the current client(s) stock portfolio from the portfolio table in the database.
    """

    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
    db = SQLDatabase.from_uri('sqlite:///portfolio_allocations.db')
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    result = agent_executor.invoke(query)
    

    client = get_numbers(query)
    for i in client:
        response_schemas = [
        ResponseSchema(name=f'Client {i}', description=f"A dictionary containing the stock tickers and their percentage allocations for each client {i}.")
        ]
      
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    format_instructions = output_parser.get_format_instructions()
    
    prompt = PromptTemplate(
    template="create a new key, value pair for every client mentioned in the question.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
    )

    chain = prompt | llm | output_parser

    ans = chain.invoke({"question": result})

    return ans


    
    


@tool
def portfolio_allocation(query : str):
    """
    Based on the allocation stratergy provided as a query, context on the comapany's recent financials from their 10-k reports
    and their up-to-date status from an online search, change the allocation in the
    client stock portfolios in the SQL database.
    """
 
    template = """
    You are a helpful assistant who is going to allocate different weights based on context and the invest stratergy that user wants.
    In order to retreieve the context, use the search_tool and the create_embedding tool to search 'Tell me about the fininical siutuation of
    AAPL, MSFT, NVIDIA'. Combining this with the allocation stratergy that the user picks, redistribute the allocation percentages. Give exact figures 
    as to how this is to be used for each stock. You should also give a detailed explanation as to why you came up with these figures. Use the context
    to provide statistics in order to boost your reasoning. Make sure the reasoning is also returned in the final answwer.

    Query : {query}
    {answer}
    """
   
    examples = [
    {
        'query': 'Reallocte the portfolio for client 1 with a balanced strategy',
        'answer': 'The stock allocations for client_id 1 equal across all stocks, totalling 100.'
    },

    {
        'query': 'Reallocte the portfolio for client 1 with a risk-based strategy',
        'answer': """
                        Allocating more to stocks with lower perceived risk. 
                        From my 10k data and online search, Company 1 has strong financials
                        and minimal risk but Comapany 2 has more risk. As a result, the stock
                        allocation fror client_id 1 would have a higher percentage for Company 1
                        and a lower percentage for Company 2. Company 3 would make up the rest of the portfolio.
                        """
    },

    {
        'query': 'Reallocte the portfolio for client 1 with a return-based strategy',
        'answer' : """
                        Allocating more to stocks with higher expected returns.
                        From my 10k data and online search, Company 3 has higher expected growth rates and returns
                        whereas Company 2 has a declining industry position and poor management quality. As a result, the stock
                        allocation fror client_id 1 would have a high percentage for Company 3
                        and a low percentage for Company 2 and Company 1 would make up the rest of the portfolio.
                        """
    }
    ]
    c1 = create_embeddings("what is the current financial outlook of AAPL, MSFT, NVIDIA")
    c2 = search_tool("what is the current financial outlook of AAPL, MSFT, NVIDIA")

    context = c1 + c2

    example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template=template
    )

    prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Query: {input}, Context: {context}",
    input_variables=["input", "context"],
    )

    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)

    chain = prompt | llm

    #print(f"context: {context}")
    ans = chain.invoke({'input' : query, 'context':context})
    
    query_database(f"set the stock allocations for the client and percentages from {ans.content} provide the sql query to do so and execute it on the database")
    
    return ans.content

    
tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Uses the content from the file path to create embeddings on the inputted data. Using the query provided, perform a similarity search on the query and return the output. If you can not provide a response,say I don't know rather than providing an incorrect response."),
         Tool(name='query_database', func=query_database, description= "This function queries the SQL database based on the query that the user provides to provide insight into the current client stock portfolio"),
         Tool(name='portfolio_allocation', func=portfolio_allocation, description='This function uses context from the vector database and online searches as well as the investment stratergy the user specifies to determine the stock allocation splits')]

In [5]:
template = """
You need to use the tools to find the appropriate action to take. 

If you are searching for information use the create_emebddings tool FIRST.

Only use the query_database and portfolio_allocation tools when the term "clients" is mentioned in the query .
if only the query_database is used, the output has to be EXACTLY as it is in the function.
and if further queries are presented alongside the term "clients", then use all tools available.

Try to get relevant responses within our internal documents. If you get relevant
responses, return them to the user. 

If you do not get relevent results from the embeddings, use the search_tool to get responses from the internet.

If you still can't find an answer from an online search, 
please do not make up an answer and return 'Sorry I cannot answer your query' instead.

If performing portfolio_allocation, use the tools "search_tool" and "create_embeddings" 
to get context around AAPL, MSFT, NVIDIA's financial situation. 
Use a query with these tools that states: "Tell me about the company's financial situation".

You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

If your thought before the final answer includes the output from the portfolio_allocation tool, give the response by including ACTUAL percentages totalling to 100. 

If your thought before the final answer uses the query_database tool, treat the tool as a function and return a dictionary output as the final answer.

Final Answer: the final answer to the original input question

Begin!


Question: {input}

Thought:{agent_scratchpad}
"""


# Define the query and create the prompt
#query = 'return the values for client_id 1 in the sql database and also tell me the current apple stock price'
query = 'Tell me the todays Apple stock price'
#query = 'Return me the stock allocation for client 5'
# query = 'Return me the stock allocation for client 8'
#query = 'Return me the stock allocation for every client'
#query = 'give me all the stock allocations from all clients up to client 3'
#query = "The Company continues to develop new technologies to enhance existing products and services and to expand the range of its offerings through research and development RD licensing of intellectual property and acquisition of thirdparty businesses and technology"
# query = "Give me a sentence from the apple 10-k report"
#query = "what is the net sales of iPhones in 2021"
#query =  "add a new client to the database with random stock allocations, provide the sql query to do so and execute it"
#query =  "set the stock allocations for client_id 1 to 5,5,90. provide the sql query to do so and execute it on the database" #remeber to pull numbers from another query into this framework
#query = 'give me all the stock allocations from all clients up to client 3'
# query = 'give me all the stock allocations for all clients up to client 3'
# query = 'Reallocate the stocks for client 3 using a balanced stratergy'

prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])

c:\GenAIProject\GenAIGroupProject\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




> Entering new AgentExecutor chain...
I should use the search_tool to find the current Apple stock price.

Action: search_tool
Action Input: "Apple stock price today"Get the latest Apple (AAPL) stock price, quote, news and history on Nasdaq. See the bid, ask, volume, 52 week range, earnings surprises, PEG ratio and more for AAPL. Get the latest stock price, change, volume and historical data for Apple (NASDAQ: AAPL) as of May 6, 2024. See the stock performance and charts for the past year and compare with other dates. Stock analysis for Apple Inc (AAPL:NASDAQ GS) including stock price, stock chart, company news, key statistics, fundamentals and company profile. View Apple Inc. AAPL stock quote prices, financial information, real-time forecasts, and company news from CNN. View Apple Inc AAPL investment & stock information. Get the latest Apple Inc AAPL detailed stock quotes, stock data, Real-Time ECN, charts, stats and more. Apple - AAPL - Stock Price Today - ZacksI should now have th